In [2]:
import pandas as pd
import numpy as np

import duckdb

# Creation

In [3]:
right = pd.DataFrame({
    'id':[3,4,5], 
    'val_right':[300,300,500], 
    'val2_right':[-7,np.nan,-3]})

left = pd.DataFrame({
    'id': [1,2,3,4,5,6], 
    'val_left': [20,10,30,40,5,20], 
    'val2_left': [-3,np.nan,-2.1,-1, 0.5, 0.25],
    'str_left': ['hello','there','dude',np.nan,'!!!',np.nan],
    'date_string': ['2025-08-01T13:45:00',
                    '2025-08-02T11:47:00',
                    '2025-08-05T16:42:00',
                    '2025-08-09T09:45:00',
                    '2025-08-03T09:45:00',
                    '2025-08-05T16:42:00'],
    'grp': ['A','A','A','B','B','A'],
})
left['date_type'] = pd.to_datetime(left['date_string'], errors='coerce')

print(left.dtypes)
left

id                      int64
val_left                int64
val2_left             float64
str_left               object
date_string            object
grp                    object
date_type      datetime64[ns]
dtype: object


,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [4]:
right

,id,val_right,val2_right
0,3,300,-7.0
1,4,300,NaN
2,5,500,-3.0


# Filtering

In [5]:
right[right['val2_right'] > -2]

,id,val_right,val2_right


In [12]:
right_table = right.copy() # looks like 'right' (and prob 'left' too?) are reserved? I get a syntax error when I use 'right' in the SQL statement

duckdb.sql("""SELECT * FROM right_table WHERE val2_right > -2""").df()

,id,val_right,val2_right


In [193]:
right.sort_values('val2_right')

,id,val_right,val2_right
0,3,300,-7.0
2,5,500,-3.0
1,4,300,NaN


In [16]:
# w/o the '.df()' after the .sql, we just get a query obj that prints as a table like so
duckdb.sql("SELECT * FROM right_table ORDER BY val2_right")

┌───────┬───────────┬────────────┐
│  id   │ val_right │ val2_right │
│ int64 │   int64   │   double   │
├───────┼───────────┼────────────┤
│     3 │       300 │       -7.0 │
│     5 │       500 │       -3.0 │
│     4 │       300 │       NULL │
└───────┴───────────┴────────────┘

In [17]:
duckdb.sql("SELECT * FROM right_table ORDER BY val2_right").df()

,id,val_right,val2_right
0,3,300,-7.0
1,5,500,-3.0
2,4,300,NaN


In [194]:
right.sort_values('val2_right', ascending=False)

,id,val_right,val2_right
2,5,500,-3.0
0,3,300,-7.0
1,4,300,NaN


In [20]:
duckdb.sql("SELECT * FROM right_table ORDER BY val2_right DESC").df()

,id,val_right,val2_right
0,5,500,-3.0
1,3,300,-7.0
2,4,300,NaN


In [23]:
right.sort_values(['val_right', 'val2_right'], ascending=[False, False]) 

,id,val_right,val2_right
2,5,500,-3.0
0,3,300,-7.0
1,4,300,NaN


In [24]:
duckdb.sql("SELECT * FROM right_table ORDER BY val_right DESC, val2_right DESC").df()

,id,val_right,val2_right
0,5,500,-3.0
1,3,300,-7.0
2,4,300,NaN


In [25]:
left_table = left.copy()

In [26]:
left['val_left']

0    20
1    10
2    30
3    40
4     5
5    20
Name: val_left, dtype: int64

In [33]:
duckdb.sql("SELECT val_left FROM left_table").df()['val_left']

0    20
1    10
2    30
3    40
4     5
5    20
Name: val_left, dtype: int64

In [197]:
left['val_left'].reset_index()

,index,val_left
0,0,20
1,1,10
2,2,30
3,3,40


In [198]:
left[['val_left']]

,val_left
0,20
1,10
2,30
3,40


In [34]:
left.query('val_left >= 20')

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [35]:
duckdb.sql("SELECT * FROM left_table WHERE val_left >= 20").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
2,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
3,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [200]:
left.iloc[1]

id                               2
val_left                        10
val2_left                      NaN
str_left                     there
date_string    2025-08-02T11:47:00
date_type      2025-08-02 11:47:00
Name: 1, dtype: object

In [201]:
left.iloc[1]['val_left']

np.int64(10)

In [202]:
left.loc[:, 'val2_left']

0   -3.0
1    NaN
2   -2.1
3   -1.0
Name: val2_left, dtype: float64

In [203]:
left.loc[3, :]

id                               4
val_left                        40
val2_left                     -1.0
str_left                       NaN
date_string    2025-08-09T09:45:00
date_type      2025-08-09 09:45:00
Name: 3, dtype: object

In [204]:
left

,id,val_left,val2_left,str_left,date_string,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,2025-08-02 11:47:00
2,3,30,-2.1,dude,2025-08-05T16:42:00,2025-08-05 16:42:00
3,4,40,-1.0,NaN,2025-08-09T09:45:00,2025-08-09 09:45:00


In [37]:
left[
    (left['val_left'] <= 20) & 
    (left['val2_left'].notnull())
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [36]:
duckdb.sql("SELECT * FROM left_table WHERE val_left <= 20 AND val2_left IS NOT NULL").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [206]:
left[
    (left['val_left'] >= 30) | 
    (left['val2_left'].isnull())
]

,id,val_left,val2_left,str_left,date_string,date_type
1,2,10,NaN,there,2025-08-02T11:47:00,2025-08-02 11:47:00
2,3,30,-2.1,dude,2025-08-05T16:42:00,2025-08-05 16:42:00
3,4,40,-1.0,NaN,2025-08-09T09:45:00,2025-08-09 09:45:00


In [41]:
left.dropna()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
4,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00


In [40]:
left.dropna(subset=['val_left'])

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [42]:
duckdb.sql("SELECT * FROM left_table WHERE val_left IS NOT NULL").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [43]:
left.dropna(subset=['val2_left'])

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [44]:
left.fillna(1000)

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,1000.00,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,1000,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,1000,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [45]:
left.fillna({'val2_left':1000})

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,1000.00,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [46]:
left[
    left['str_left'] == 'hello'
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00


In [48]:
duckdb.sql("SELECT * FROM left_table WHERE str_left = 'hello'").df

<bound method pybind11_detail_function_record_v1_system_libcpp_abi1.df of ┌───────┬──────────┬───────────┬──────────┬─────────────────────┬─────────┬─────────────────────┐
│  id   │ val_left │ val2_left │ str_left │     date_string     │   grp   │      date_type      │
│ int64 │  int64   │  double   │ varchar  │       varchar       │ varchar │    timestamp_ns     │
├───────┼──────────┼───────────┼──────────┼─────────────────────┼─────────┼─────────────────────┤
│     1 │       20 │      -3.0 │ hello    │ 2025-08-01T13:45:00 │ A       │ 2025-08-01 13:45:00 │
└───────┴──────────┴───────────┴──────────┴─────────────────────┴─────────┴─────────────────────┘
>

Remember to likely always include 'na=False' in the args to contains and the like, because without it, the comparison will fail with a 'ValueError: Cannot mask with non-boolean array containing NA / NaN values' message. With 'na=False', then nulls are treated as false in the resulting boolean array.

In [49]:
left[
    #left['str_left'].str.contains('he') # causes exception
    left['str_left'].str.contains('he', na=False)
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00


In [56]:
duckdb.sql("FROM left_table WHERE str_left LIKE '%he%'").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00


In [50]:
left[
    left['str_left'].str.contains('ER', na=False, case=False)
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00


In [58]:
duckdb.sql("FROM left_table WHERE str_left ILIKE '%ER%'").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00


In [51]:
left[
    left['val_left'].between(15, 35)
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [59]:
duckdb.sql("FROM left_table WHERE val_left BETWEEN 15 AND 35").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [52]:
left[
    left['val_left'].isin([20, 40])
]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [60]:
duckdb.sql("FROM left_table WHERE val_left IN (20, 40)").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [53]:
left

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [62]:
left[left['date_string'] > '2025-08-04'] # works i think because this is the str order

,id,val_left,val2_left,str_left,date_string,grp,date_type
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [61]:
duckdb.sql("FROM left_table WHERE date_string > '2025-08-04'").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
1,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [63]:
left[left['date_type'] > '2025-08-04'] # i think '2025-08-04' is cast to a date

,id,val_left,val2_left,str_left,date_string,grp,date_type
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [64]:
duckdb.sql("FROM left_table WHERE date_type > '2025-08-04'").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
1,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [65]:
(
    left['date_type'].dt.year, 
    left['date_type'].dt.month, 
    left['date_type'].dt.day
)
# no dt.week

(0    2025
 1    2025
 2    2025
 3    2025
 4    2025
 5    2025
 Name: date_type, dtype: int32,
 0    8
 1    8
 2    8
 3    8
 4    8
 5    8
 Name: date_type, dtype: int32,
 0    1
 1    2
 2    5
 3    9
 4    3
 5    5
 Name: date_type, dtype: int32)

In [67]:
duckdb.sql("SELECT DATE_PART('year', date_type), DATE_PART('month', date_type), DATE_PART('day', date_type), DATE_PART('week', date_type) FROM left_table").df()

,"date_part('year', date_type)","date_part('month', date_type)","date_part('day', date_type)","date_part('week', date_type)"
0,2025,8,1,31
1,2025,8,2,31
2,2025,8,5,32
3,2025,8,9,32
4,2025,8,3,31
5,2025,8,5,32


In [68]:
left['date_type'].dt.floor('D') # one way to drop any times (don't have here because these are datetime64 objs)

0   2025-08-01
1   2025-08-02
2   2025-08-05
3   2025-08-09
4   2025-08-03
5   2025-08-05
Name: date_type, dtype: datetime64[ns]

In [69]:
duckdb.sql("SELECT DATE_TRUNC('day', date_type) FROM left_table").df()

,"date_trunc('day', date_type)"
0,2025-08-01
1,2025-08-02
2,2025-08-05
3,2025-08-09
4,2025-08-03
5,2025-08-05


In [222]:
left['date_type'].dt.normalize() # another way

0   2025-08-01
1   2025-08-02
2   2025-08-05
3   2025-08-09
Name: date_type, dtype: datetime64[ns]

In [223]:
left['date_type'].dt.date # not datetime64, just vanilla python datetime.date objs

0    2025-08-01
1    2025-08-02
2    2025-08-05
3    2025-08-09
Name: date_type, dtype: object

In [224]:
# left['date_type'].dt.floor('MS') # for trunc, this or with 'M' gives errors - see next cell 

In [225]:
left['date_type'].dt.to_period('M').dt.to_timestamp()

0   2025-08-01
1   2025-08-01
2   2025-08-01
3   2025-08-01
Name: date_type, dtype: datetime64[ns]

In [ ]:
duckdb.sql("SELECT DATE_TRUNC('month', date_type) FROM left_table").df()

,"date_trunc('month', date_type)"
0,2025-08-01
1,2025-08-01
2,2025-08-01
3,2025-08-01
4,2025-08-01
5,2025-08-01


In [226]:
left['date_type'].dt.to_period('W').dt.to_timestamp() # also supports week

0   2025-07-28
1   2025-07-28
2   2025-08-04
3   2025-08-04
Name: date_type, dtype: datetime64[ns]

Remember to use Decimal type for exactness, since floating point can get comparisons wrong. np.isclose is an option, but requires a mask since I think it returns a boolean array?

In [71]:
np.isclose(left['val2_left'], -2.0) 

array([False, False, False, False, False, False])

In [72]:
left[np.isclose(left['val2_left'], -2.1)]

,id,val_left,val2_left,str_left,date_string,grp,date_type
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [73]:
duckdb.sql("FROM left_table WHERE val2_left = -2.1").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [231]:
left[left['val2_left'] == -2.1] # looks like this one works, but we shouldn't rely on it?

,id,val_left,val2_left,str_left,date_string,date_type
2,3,30,-2.1,dude,2025-08-05T16:42:00,2025-08-05 16:42:00


# Joins

In [233]:
left

,id,val_left,val2_left,str_left,date_string,date_type
0,1,20,-3.0,hello,2025-08-01T13:45:00,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,2025-08-02 11:47:00
2,3,30,-2.1,dude,2025-08-05T16:42:00,2025-08-05 16:42:00
3,4,40,-1.0,NaN,2025-08-09T09:45:00,2025-08-09 09:45:00


In [234]:
right

,id,val_right,val2_right
0,3,300,-7.0
1,4,300,NaN
2,5,500,-3.0


In [76]:
pd.merge(left, right, on='id') # looks like how='inner' is the default

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300,-7.0
1,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300,NaN
2,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500,-3.0


In [75]:
duckdb.sql("FROM left_table INNER JOIN right_table ON left_table.id = right_table.id").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,id_1,val_right,val2_right
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3,300,-7.0
1,4,40,-1.0,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,4,300,NaN
2,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,5,500,-3.0


In [77]:
pd.merge(left, right, how='left')

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,NaN,NaN
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,NaN,NaN
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300.0,-7.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300.0,NaN
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500.0,-3.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,NaN,NaN


In [78]:
duckdb.sql("FROM left_table LEFT JOIN right_table ON left_table.id = right_table.id").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,id_1,val_right,val2_right
0,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3,300,-7.0
1,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,4,300,NaN
2,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,5,500,-3.0
3,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,<NA>,<NA>,NaN
4,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,<NA>,<NA>,NaN
5,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,<NA>,<NA>,NaN


In [79]:
pd.merge(left, right, how='right', indicator=True)

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right,_merge
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300,-7.0,both
1,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300,NaN,both
2,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500,-3.0,both


In [80]:
pd.merge(left, right, how='right', suffixes=['_left','_right'])

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300,-7.0
1,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300,NaN
2,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500,-3.0


In [81]:
pd.merge(left, right, how='outer', indicator=True)

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right,_merge
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,NaN,NaN,left_only
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,NaN,NaN,left_only
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300.0,-7.0,both
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300.0,NaN,both
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500.0,-3.0,both
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,NaN,NaN,left_only


Remember there's also a 'validate' param that takes 'one_to_many' (keys can repeat on the right), 'one_to_one' (keys can't repeat on either side), 'many_to_one' (keys can repeat on the left), and 'many_to_many' (keys can repeat on both sides). If the dataframe(s) violate these, then pandas will raise an exception. 

In [82]:
pd.merge(left, right, how='cross')

,id_x,val_left,val2_left,str_left,date_string,grp,date_type,id_y,val_right,val2_right
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,3,300,-7.0
1,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,4,300,NaN
2,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,5,500,-3.0
3,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,3,300,-7.0
4,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,4,300,NaN
5,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,5,500,-3.0
6,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3,300,-7.0
7,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4,300,NaN
8,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,5,500,-3.0
9,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,3,300,-7.0


Also, you can join on the DF index with 'left.join(right, how='left').

In [83]:
pd.concat([left, right])

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right
0,1,20.0,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,NaN,NaN
1,2,10.0,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,NaN,NaN
2,3,30.0,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,NaN,NaN
3,4,40.0,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,NaN,NaN
4,5,5.0,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,NaN,NaN
5,6,20.0,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,NaN,NaN
0,3,NaN,NaN,NaN,NaN,NaN,NaT,300.0,-7.0
1,4,NaN,NaN,NaN,NaN,NaN,NaT,300.0,NaN
2,5,NaN,NaN,NaN,NaN,NaN,NaT,500.0,-3.0


Can do 'pd.concat([left, right]).drop_duplicates() for UNION ALL.

Also, you can not use pd.merge and instead use left.merge(right...) in general.

In [84]:
# EXCEPT, anti-join - rows in left that aren't in right
m = pd.merge(left, right, how='left', indicator=True)
m[m['_merge'] == 'left_only']
# m[m['_merge'] == 'left_only'].drop(columns='_merge') # drop optional, to get rid of _merge col 

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right,_merge
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,NaN,NaN,left_only
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,NaN,NaN,left_only
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,NaN,NaN,left_only


In [85]:
# above in contrast to the inner join
pd.merge(left, right, how='inner', indicator=True)

,id,val_left,val2_left,str_left,date_string,grp,date_type,val_right,val2_right,_merge
0,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,300,-7.0,both
1,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,300,NaN,both
2,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,500,-3.0,both


Also, you can do semi joins and anti joins, not just by the sql approach (where for a an anti join you do a left join and then look at the right table nulls and filter to just those), but by using .isin on the boolean series.

Remember that the thing that makes semi and anti joins diff (from inner/outer joins) is that you only keep the cols in the orig (for ex left) table - you don't bring in any cols from the other table.

The pandas impls are easy - they use .isin as part of the boolean array for selection of rows from the left data set. 

For the semijoin the boolean series is `left['id'].isin(right['id'])` - i.e., it's a boolean series where the only True values are those where the left and right id fields match. 

For the anti-join, it's exactly the same with one character difference: I just element-wise negate the exact same boolean series as above py pre-pending it with a tilde, so I use `~left['id'].isin(right['id'])`.

In [86]:
# semi-join - return rows in left table where no match is in the right table
# SQL approach: left join and then filter to rows where right table field(s) are null
# (and the pandas impl above does the same thing or could using .isnull)
left[left['id'].isin(right['id'])]

,id,val_left,val2_left,str_left,date_string,grp,date_type
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00


In [87]:
# anti-join - return rows in the left table that DO have a match in the right
# SQL approach: inner join+distinct on tbl A fields, or select a.* from a where exists (select 1 from b where a.key=b.key)
left[~left['id'].isin(right['id'])]

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


## Remember diff between pd.merge and SQL join because null handling

Pandas treats np.nan as equal when joining, which SQL doesn't do: in SQL NULL is never equal to anything, including another NULL. That is, in SQL NULL != NULL, but in pandas np.nan == np.nan (at least for merge purposes).

In practice, this means that in pandas merges i might end up with extra rows, where both sides have nulls in join keys, and in pandas might mean I want to do things that I do w/ SQL with 'is null' by instead using indicator=True in the join and the resulting _merge field.

# Aggregation, GROUP BY, HAVING

One key thing is that in pandas groupby isn't tied to only resulting in one row per group - instead, it's just about the grouping. What you get out - one row per group or one row per input row (or even other shapes) is defined by what you do with the result of the groupby (which is itself an object of type DataFrameGroupBy object).

In [88]:
left.groupby(['grp'])

The 'agg' function, or the syntatic shorthand of specifying a column and then an agg function like sum or count, collapses to a row per group.

In [89]:
left

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [90]:
left.groupby(['grp'])['val_left'].sum() # returns a series

grp
A    80
B    45
Name: val_left, dtype: int64

In [ ]:
duckdb.sql("SELECT SUM(val_left) AS sum FROM left_table GROUP BY grp").df()

,sum
0,80.0
1,45.0


In [94]:
duckdb.sql("SELECT SUM(val_left) AS sum FROM left_table GROUP BY grp").df()['sum']

0    45.0
1    80.0
Name: sum, dtype: float64

In [95]:
left.groupby(['grp'], as_index=False)['val_left'].sum() # as_index makes it return a df, with A,B in the df

,grp,val_left
0,A,80
1,B,45


In [97]:
duckdb.sql("SELECT grp, SUM(val_left) AS sum FROM left_table GROUP BY grp").df()

,grp,sum
0,A,80.0
1,B,45.0


In [98]:
left.groupby(['grp'])['val_left'].sum().reset_index() # or use reset_index afterwards

,grp,val_left
0,A,80
1,B,45


In [99]:
left.groupby(['grp'])['val_left'].sum().reset_index(drop=True) 
# use drop to get rid of the A,B index but not move it into the df... giving a series
# I think if there's only one column left

0    80
1    45
Name: val_left, dtype: int64

In [100]:
left.groupby(['grp']).agg(total=('val_left', 'sum')) # df with A, B in index

,total
grp,
A,80
B,45


In [101]:
left.groupby(
    ['grp'], as_index=False
).agg(
    total=('val_left', 'sum')
) # df with A,B out of index

,grp,total
0,A,80
1,B,45


Can do multiple fields w/ .agg w/ names, cleanly. To remember is the format for each row, with as many rows as you want aggregation calcs, which is 

`output_col_name=('input_col_name', 'agg func name')` - all three are strings w/ quotes, and the right two are a tuple

so:

`total=('val_left', 'sum')`

In [102]:
left.groupby(
    ['grp'], as_index=False
).agg(
    total=('val_left', 'sum'),
    count=('val_left', 'count'),
    size=('val_left', 'size'),
    avg=('val_left', 'mean'),
    count_col_with_nulls=('val2_left', 'count'), # count doesn't count nulls
    size_col_with_nulls=('val2_left', 'size'), # size DOES count nulls
)

,grp,total,count,size,avg,count_col_with_nulls,size_col_with_nulls
0,A,80,4,4,20.0,3,4
1,B,45,2,2,22.5,2,2


In [105]:
duckdb.sql("""
    SELECT
        grp,
        SUM(val_left),
        COUNT(val_left),
        COUNT(*),
        AVG(val_left),
        COUNT(val2_left),
        COUNT(*)
    FROM left_table
    GROUP BY
        grp""").df()

,grp,sum(val_left),count(val_left),count_star(),avg(val_left),count(val2_left),count_star()_1
0,A,80.0,4,4,20.0,3,4
1,B,45.0,2,2,22.5,2,2


In [106]:
left.groupby(
    ['grp']
).agg(
    total=('val_left', 'sum'),
    size=('val_left', 'size')
).reset_index(drop=True) # totally get rid of A,B

,total,size
0,80,4
1,45,2


To do the equivalent of HAVING, group and aggregate to row-per-group and then filter the resulting aggregated df.

In [107]:
grouped = left.groupby(['grp']).agg(total=('val_left', 'sum')).reset_index()
grouped[grouped['total'] > 50]

,grp,total
0,A,80


In [109]:
duckdb.sql("SELECT grp, SUM(val_left) AS sum_val_left FROM left_table GROUP BY grp HAVING sum_val_left > 50").df()

,grp,sum_val_left
0,A,80.0


# Window function-type stuff

Basic idea is to use groupby again, but with functions that are NOT aggregation functions - i.e., they're functions that don't collapse to a row per group or are used with 'transform' instead of 'agg' (the latter like 'sum', 'count', 'mean', etc.). There are also functions that are typically used for things that match the typical window uses, including cumcount, cumsum (there are also cummin, cummax, cumprod), 'shift' (for lag/lead), and 'rank' (with param of 'method=min' for the equiv of RANK and method='dense' for DENSE_RANK/no gaps). These - i think whether it's .transform or a function like .shift - returns a series that you can then append to the dataframe with df['new_row_name'] = ...

The other thing to keep in mind is to typically sort before the groupby because the funcs above don't have their own notion of ORDER BY (rank does have ascending, but i think it might be diff).   

In [110]:
left

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [118]:
left.groupby(['grp'])['val_left'].sum()

grp
A    80
B    45
Name: val_left, dtype: int64

In [111]:
# non-cum sum - sum of all items in the group
left_copy = left.copy() # since I want to show the appending to the df pattern w/o changing the source

left_copy['grp_sum'] = left.groupby(['grp'])['val_left'].transform('sum')
left_copy

# transform doesn't support the named tuple syntax like we use in agg, so do the above

,id,val_left,val2_left,str_left,date_string,grp,date_type,grp_sum
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,80
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,80
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,45
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,45
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80


In [113]:
duckdb.sql("SELECT *, SUM(val_left) OVER (PARTITION BY grp) AS grp_sum FROM left_table").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,grp_sum
0,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,45.0
1,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,45.0
2,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,80.0
3,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,80.0
4,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80.0
5,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80.0


In [116]:
# for lag/lead, first sort by partition key(s) and then by the val key in desired order
# then shift(-1) or shift(1) to get lag or lead vals
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val_left'])
left_copy['lag_val_left'] = left_copy.groupby(['grp'])['val_left'].shift(1)
left_copy['lead_val_left'] = left_copy.groupby(['grp'])['val_left'].shift(-1)
left_copy.sort_values(['id'])

,id,val_left,val2_left,str_left,date_string,grp,date_type,lag_val_left,lead_val_left
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,10.0,20.0
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,NaN,20.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,20.0,NaN
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,5.0,NaN
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,NaN,40.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,20.0,30.0


In [117]:
duckdb.sql("""
        SELECT
            *,
            LAG(val_left) OVER (PARTITION BY grp ORDER BY val_left) AS lag_val_left,
            LEAD(val_left) OVER (PARTITION BY grp ORDER BY val_left) AS lead_val_left,  
        FROM left_table
        ORDER BY 
            id""").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,lag_val_left,lead_val_left
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,10,20
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,<NA>,20
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,20,<NA>
3,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,5,<NA>
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,<NA>,40
5,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,20,30


With .shift, I found the + and - confusing. To get LAG/prev row, you shift by a positive number; to get LEAD/following row, you shift by a negative number. There's explanation I guess in that it's talking about temporal position, so shift(1) means to shift to what was 'before' in time assuming the prev row is earlier in time. I guess.

In [123]:
left_copy = left.copy()

# note that cumcount doesn't have to have a particular col selected just before
# i.e., it doesn't have to be df.groupby(['grp']['col_name'].cumcount
# but it can be... so maybe I just always think w/ transform and other window functions
# to specify the column after the groupby

left_copy = left_copy.sort_values(['grp','val_left'])
left_copy['row_number'] = left_copy.groupby(['grp'])['val_left'].cumcount() + 1
# left_copy['row_number'] = left_copy.groupby(['grp'])['val_left'].cumcount() + 1
left_copy.sort_values(['grp','val_left'])

,id,val_left,val2_left,str_left,date_string,grp,date_type,row_number
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2


In [125]:
duckdb.sql("SELECT *, ROW_NUMBER() OVER (PARTITION BY grp ORDER BY val_left) FROM left_table ORDER BY grp, val_left").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,row_number() OVER (PARTITION BY grp ORDER BY val_left)
0,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1
1,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3
3,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1
5,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2


In [126]:
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val2_left'])
left_copy['row_number'] = left_copy.groupby(['grp']).cumcount() + 1
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,row_number
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,1
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,3
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,4
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,1
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,2


All of the next four items show using normal .rank(method='min') which is the equiv of SQL RANK and skips numbers after ties. The differences between the four are whether the data frame is sorted before using .rank and whether there's an 'ascending' in the rank call itself. 

Bottom line, I'm going to keep w/ the practice of sorting the DF first just to be consistent, but I don't think it's necessary because it looks like the .rank call, using the optional 'ascending' param inside the rank col, takes care of the ordering and the rank results are independent of the order of the DF before the call. This is not the case for at least some, maybe all, of the other functions we'd use, like those above.

In [127]:
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val_left'])
left_copy['rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='min')
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1.0
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2.0


In [128]:
duckdb.sql("SELECT *, RANK() OVER (PARTITION BY grp ORDER BY val_left) FROM left_table ORDER BY grp, val_left").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank() OVER (PARTITION BY grp ORDER BY val_left)
0,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1
1,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2
3,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1
5,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2


In [129]:
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val_left'], ascending=[True, False])
left_copy['rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='min')
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4.0
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2.0
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1.0


In [130]:
left_copy = left.copy()

# left_copy = left_copy.sort_values(['grp','val_left'], ascending=[True, False])
left_copy['rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='min')
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2.0
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2.0


In [131]:
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val_left'], ascending=[True, True])
left_copy['rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='min', ascending=False)
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,4.0
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,1.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,2.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,1.0


In [132]:
# DENSE_RANK (and normal RANK for comparison)
left_copy = left.copy()

left_copy = left_copy.sort_values(['grp','val_left'])
left_copy['rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='min')
left_copy['dense_rank'] = left_copy.groupby(['grp'])['val_left'].rank(method='dense')
left_copy

,id,val_left,val2_left,str_left,date_string,grp,date_type,rank,dense_rank
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1.0,1.0
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2.0,2.0
5,6,20,0.25,NaN,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2.0,2.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4.0,3.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1.0,1.0
3,4,40,-1.00,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2.0,2.0


In [133]:
duckdb.sql("""
        SELECT
            *, 
            RANK() OVER (PARTITION BY grp ORDER BY val_left) AS rnk,
            DENSE_RANK() OVER (PARTITION BY grp ORDER BY val_left) AS dense_rnk
        FROM left_table
        ORDER BY
           grp,
           val_left           
        """).df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,rnk,dense_rnk
0,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,1,1
1,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2,2
2,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,2,2
3,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,4,3
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,1,1
5,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,2,2


There's stuff in the gpt-5 doc about window frame things, with pandas .rolling() and .expanding() and .ewm(), but I'm not going to dig into them.

But I will try to refresh my memory with a SQL implementation, since it's simpler to remember

In [135]:
duckdb.sql("""
        SELECT 
            *,
            SUM(val_left) OVER (PARTITION BY grp) AS sum_of_group,
            SUM(val_left) OVER (PARTITION BY grp ORDER BY date_type) AS rolling_sum_in_group
        FROM left_table
        """).df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,sum_of_group,rolling_sum_in_group
0,1,20,-3.00,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,80.0,20.0
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,80.0,30.0
2,3,30,-2.10,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80.0,80.0
3,6,20,0.25,None,2025-08-05T16:42:00,A,2025-08-05 16:42:00,80.0,80.0
4,5,5,0.50,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,45.0,5.0
5,4,40,-1.00,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,45.0,45.0


# nunique, and top n per group (with and without nlargest)

First off, for the number of distinct values...

In [136]:
left['val_left'].nunique()

5

In [137]:
duckdb.sql("SELECT COUNT(DISTINCT val_left) FROM left_table").df()

,count(DISTINCT val_left)
0,5


For the top N per group, with cumcount:

In [146]:
left_copy = left.copy()

left_copy.sort_values(['grp','val_left'], ascending=[True,False])
left_copy['row_number'] = left_copy.groupby(['grp']).cumcount() + 1
left_copy[left_copy['row_number'] <= 2]

,id,val_left,val2_left,str_left,date_string,grp,date_type,row_number
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,1
1,2,10,NaN,there,2025-08-02T11:47:00,A,2025-08-02 11:47:00,2
3,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00,1
4,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,2


In [147]:
# needs a CTE to define the row number, since the where clause can't contain (or reference) window functions created in the same query
duckdb.sql("""
        WITH aug AS (
            SELECT
                *,
                ROW_NUMBER() OVER (PARTITION BY grp ORDER BY val_left DESC) AS row_num 
            FROM left_table
        )
        SELECT * FROM aug WHERE row_num <= 2
        """).df()

,id,val_left,val2_left,str_left,date_string,grp,date_type,row_num
0,4,40,-1.0,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00,1
1,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00,2
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00,1
3,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00,2


nlargest(n, columns) returns the top n rows with the largest vals in the specified cols - it returns the whole rows, and is faster than sorting the entire df if we only want the top rows (i.e., we could sort by those cols and then take head(n) to get the same result). 

In [326]:
top2 = left.nlargest(2, 'val_left')
top2

,id,val_left,val2_left,str_left,date_string,grp,date_type
3,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00


In [143]:
duckdb.sql("FROM left_table ORDER BY val_left DESC LIMIT 2").df()

,id,val_left,val2_left,str_left,date_string,grp,date_type
0,4,40,-1.0,None,2025-08-09T09:45:00,B,2025-08-09 09:45:00
1,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00


The above is not grouped. To do group, prob the right way is the cell a few above where you do basically what you'd do in SQL: use a window function to add a row number or rank (depending on the exact thing you're trying to do) and then filter to only those rows.

You can do something similar also though w/ nlargest, by using .apply to run it for each group, like so, and noticing that there's no ranking info/col like there is two cells above. Just put this aside.

In [144]:
left_copy = left.copy()

left_copy.groupby(['grp'], group_keys=False).apply(lambda g: g.nlargest(2, 'val_left'))

/var/folders/32/q9lv9grs0h90h1rwfq_dm_1m0000gn/T/ipykernel_35685/1266224487.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  left_copy.groupby(['grp'], group_keys=False).apply(lambda g: g.nlargest(2, 'val_left'))


,id,val_left,val2_left,str_left,date_string,grp,date_type
2,3,30,-2.1,dude,2025-08-05T16:42:00,A,2025-08-05 16:42:00
0,1,20,-3.0,hello,2025-08-01T13:45:00,A,2025-08-01 13:45:00
3,4,40,-1.0,NaN,2025-08-09T09:45:00,B,2025-08-09 09:45:00
4,5,5,0.5,!!!,2025-08-03T09:45:00,B,2025-08-03 09:45:00


# .apply with groups

We use .agg to turn mult rows into one row (per group) and typically work off of a series as input. In contrast .apply runs a given function for each group, and the input to the function is a dataframe itself (the subset, defined by the group by fields). the function you pass to .apply can return different kinds of outputs. If you return a scalar, .apply returns a series; if you return a series, .apply concats the series; if you return a dataframe, .apply concats the dataframes (and there's more for others). Note that the concatenated series or dataframes have MultiIndexes by default, and you can pass group_keys=False to the groupby to not add these.  